# Demo: Interactive Widgets
A notebook to explore the ipywidgets interactive widgets with clojupyter kernel.

Requires clojupyter-0.3.3.-snapshot.

In [ ]:
(require '[clojupyter.widgets.alpha :as alpha])
(require '[clojupyter.widgets.ipywidgets :as ipy])
(require '[clojure.string :as s]);

The original example, as posted on [github](https://github.com/clojupyter/clojupyter/issues/74).

In [ ]:
(def IA
    (let [myfun (fn [{:keys [a b]}]
                  (str a " + " b " = " (+ a b)))
          label (ipy/label)
          slider-1 (ipy/int-slider {:value (rand-int 100)})
          slider-2 (ipy/int-slider {:value (rand-int 100)})]
      (alpha/interactive label myfun {:a slider-1, :b slider-2})))
IA

## Simple Widgets
### Text Widgets
Let's build a text widget that returns the reverse upper case of the input string.
The function bellow achieves that:

In [ ]:
(def rev-upper (comp s/upper-case (partial reduce str) reverse))

In [ ]:
(rev-upper "My text")

Passing the fn directly to the widget does not work, because the fn passed to interactive needs to take a hash-map argument.

In [ ]:
(let [t (ipy/text {:value "Hello world"})
      label (ipy/label {:value "My text widget"})]
  (alpha/interactive label rev-upper {:value t}))

We can define a wrapper fn that has the required signature

In [ ]:
(defn mapper
  [f]
  (fn [{arg :arg}] (f arg)))

and pass it to interactive instead of the original fn.

In [ ]:
(let [t (ipy/text {:value "aroma"})
      label (ipy/label)]
  (alpha/interactive label (mapper rev-upper) {:arg t}))

Password widget is basically the same as text widget, except that the input is not echoed to the user.

In [ ]:
(let [t (ipy/password {:value "top secret pwd" :description "Password"})
      label (ipy/label)]
  (alpha/interactive label (mapper identity) {:arg t}))

In [ ]:
(let [t (ipy/textarea {:value "Some lengthy description" :description "Your Text Here:"})
      label (ipy/label)]
  (alpha/interactive label (mapper #(s/replace % " " "_")) {:arg t}))

Note: *The widget above doesn't allow to resize its width.
The same is true when using a python kernel, so it's not an issue with clojupyter.
The html element does, however, change its style attributes width & height when resizing it.
Tested in Firefox 74.0 (64-bit)*

In [ ]:
(let [t (ipy/combobox {:options ["blue" "black" "green" "yellow"]:description "Pick a color"})
      label (ipy/label)]
  (alpha/interactive label (mapper identity) {:arg t}))

In [ ]:
(ipy/html {:value "<p>Hello <b>World</b></p>" :placeholder "Some HTML" :description "Some HTML"})

Example taken from: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#HTML-Math

In [ ]:
(ipy/html-math {:value "Some math and <i>HTML</i>: \\(x^2\\) and $$\\frac{x+1}{x-1}$$"})

Note: *We need to escape the backslash in order for the above example to work.*

____
### Boolean Widgets


In [ ]:
(let [w (ipy/checkbox {:description "Click Me" :value true})
      label (ipy/label)]
  (alpha/interactive label (mapper (partial str "My state: ")) {:arg w}))

In [ ]:
(let [w (ipy/toggle-button {:value false :description "Click to Activate" :icon "hand-pointer"})]
  (alpha/interactive (ipy/label) (mapper {true "Active" false "Inactive"}) {:arg w}))

In [ ]:
(ipy/valid {:value true :description "Valid!"})

___
### Numeric Widgets

In [ ]:
(let [w (ipy/int-slider {:description "Double this:" :value 20 :min -10 :max 20
                         :orientation "vertical"})
      label (ipy/label)]
  (alpha/interactive label (mapper (comp str (partial * 2))) {:arg w}))

After enforcing the specs, the label widget only works with a string value.

In [ ]:
(clojure.spec.alpha/valid?
 :clojupyter.widgets.ipywidgets/label
 {:description "",
  :_view_module "@jupyter-widgets/controls",
  :placeholder "​",
  :layout nil,
  :value 40,
  :_view_module_version "2.0.0",
  :style nil,
  :tabbable nil,
  :_view_name "LabelView",
  :_model_module "@jupyter-widgets/controls",
  :_model_name "LabelModel",
  :_dom_classes [],
  :tooltip nil,
  :_model_module_version "2.0.0"})

In [ ]:
(let [w (ipy/float-slider {:description "Square this:" :value (rand) :min 0.0 :max 1.0
                           :step 0.01 :readout_format ".2f"})
      label (ipy/label)]
  (alpha/interactive label (mapper (comp (partial format "%.3f") #(* % %))) {:arg w}))

In [ ]:
(let [w (ipy/float-log-slider {:value 2e5 :base 10.0 :min 4.0 :max 8.0 :step 0.2})]
  (alpha/interactive (ipy/label) (mapper (comp (partial format "%.2e") float (partial / 1))) {:arg w}))

**Notes:**

*The slider above kills the kernel when dragged.*

*The slider does not show up correctly. It is initialized w/ the value 2e5, but it shows up as 2.51e5.*

In [ ]:
(let [w (ipy/int-range-slider {:value [200 1800] :max 3000 :step 5})]
  (alpha/interactive (ipy/label) (mapper (comp str #(/ (+ (first %) (last %)) 2))) {:arg w}))

Note: *Passing the type fn to most (all?) widgets kills the kernel.*

In [ ]:
(ipy/float-range-slider)

In [ ]:
(ipy/int-progress {:min 10 :max 100 :value 95})

In [ ]:
(ipy/float-progress {:min 0.0 :max 10.0 :step 0.1 :value 4.8})

Note: *How does one animate the above progress bars?*

In [ ]:
(ipy/bounded-int-text {:min 0 :max 100 :step 5})

Note: *The above widget acts like a bounded-float-text if we pass a float step.*
No longer true, now that the widgets get validated.

In [ ]:
(ipy/bounded-float-text {:min 0.0 :max 10.0 :step 0.2})

In [ ]:
(ipy/int-text {:value 233})

In [ ]:
(ipy/float-text {:value 9.33 :step 0.02})

____
### Selection Widgets

In [ ]:
(ipy/dropdown {:options ["one" "two" "three"] :value "one" :index 0})

Note: *The above example fails. To fix it, one must pass the values to :_options_labels key.
The default value does not get passed to the widget.*

In [ ]:
(ipy/dropdown {:_options_labels ["one" "two" "three"] :value "one" :description "Pick a number"})

In [ ]:
(ipy/dropdown {:_options_labels ['("one" 1) '("two" 2) '("three" 3)] :value "one" :description "Pick a number"})

Note: *The above example does not work. Error: __Error displaying widget__*

In [ ]:
(ipy/radio-buttons {:options ["dark" "light"] :value "light"})

Note: *Just as dropdown, the widget fails when passing the values to :options and :values does not work as default values.*

In [ ]:
(ipy/radio-buttons {:_options_labels ["dark" "light"] :description "Theme"})

In [ ]:
(ipy/select {:options ["english" "french" "german"] :value "french"})

In [ ]:
(ipy/select {:_options_labels ["english" "french" "german"] :value "french" :description "Language"})

Note: *Just as dropdown, the widget fails when passing the values to :options and :values does not work as default values.*

In [ ]:
(ipy/selection-slider {:options ["slower" "slow" "normal" "fast" "fastest"] :value "french" :description "Speed"})

In [ ]:
(ipy/selection-slider {:_options_labels ["slower" "slow" "normal" "fast" "fastest"] :value "normal" :description "Speed"})

Note: *Just as dropdown, the widget fails when passing the values to :options and :values does not work as default values.*

In [ ]:
(ipy/selection-range-slider {:options (map list ["slower" "slow" "normal" "fast" "fastest"] (range)):value "french" :description "Speed"})

In [ ]:
(ipy/selection-range-slider {:_options_labels (map list ["slower" "slow" "normal" "fast" "fastest"] (range)):value "french" :description "Speed"})

Note: *Just as dropdown, the widget fails when passing the values to :options and :values does not work as default values.*

In [ ]:
(ipy/toggle-buttons {:options ["english" "french" "german"] :value "french" :description "Language"})

In [ ]:
(let [w (ipy/toggle-buttons {:_options_labels ["english" "french" "german"] :value "none" :description "Language"})]
  (alpha/interactive (ipy/label) (mapper identity) {:arg w}))

Note: *Just as dropdown, the widget fails when passing the values to :options and :values does not work as default values.*
Note: *Clicking on any of the above buttons kills the kernel.*

In [ ]:
(ipy/select-multiple {:options ["Guitar" "Mandolin" "Violin" "Bass"] :value "Mandolin" :description "String Instruments"})

In [ ]:
(let [w (ipy/select-multiple {:_options_labels ["Guitar" "Mandolin" "Violin" "Bass"] :value "Mandolin" :description "String Instruments"})]
  (alpha/interactive (ipy/label) (mapper identity) {:arg w}))

Note: *Just as dropdown, the widget fails when passing the values to :options and :values does not work as default values.*
Note: *The above widget has has no output.*

____
### Container Widgets

In [ ]:
(let [w (ipy/int-slider)
      p (ipy/int-progress)]
  (.watch w :key0 (fn [_ _ _ new-state] (swap! p assoc :value (:value new-state))))
  (ipy/box {:children [w p]}))

In [ ]:
(let [w (ipy/int-slider {:orientation "vertical" :value (rand-int 101)})
      p (ipy/int-progress {:orientation "vertical"})]
  (.watch w :key0 (fn [_ _ _ new-state] (swap! p assoc :value (:value new-state))))
  (ipy/h-box {:children [w p]}))

H-box looks and feels identical with a regular box widget.

In [ ]:
(let [w (ipy/int-slider)
      p (ipy/int-progress)]
  (.watch w :key0 (fn [_ _ _ new-state] (swap! p assoc :value (:value new-state))))
  (ipy/v-box {:children [w p]}))

In [ ]:
(let [w (ipy/label {:value "Hello"})
      p (ipy/label {:value "World"})]
  (ipy/accordion {:children [w p]}))

In [ ]:
(let [w (ipy/label {:value "Hello"})
      p (ipy/label {:value "World"})]
  (ipy/tab {:children [w p]}))

In [ ]:
(let [l0 (ipy/label {:value "Hello"})
      l1 (ipy/label {:value "World"})
      s (ipy/stacked {:children [l0 l1] :selected_index 0})
      w0 (ipy/radio-buttons {:_options_labels ["Stack 0" "Stack 1"] :description "Select stack"})]
  (.watch w0 :key0 (fn [_ _ _ new-state] (swap! s assoc :selected_index (:index new-state))))
  (ipy/box {:children [w0 s]}))

____
### Other Widgets

In [ ]:
(require '[clojure.java.io :as io])
(defn slurp-bytes
  [^String filename & opt]
  (let [file (io/file filename)
        buf (byte-array (.length file))]
    (with-open [fis (io/make-input-stream file (when opt (apply hash-map opt)))]
      (.read fis buf))
    buf))

In [ ]:
(let [img (slurp-bytes "../resources/clojupyter/assets/logo-64x64.png")]
  (ipy/image {:value img :format "png" :height "200" :width "120"}))

In [ ]:
(let [w (ipy/button {:description "Click Here"})]
  (alpha/interactive (ipy/label) (mapper identity) {:arg w}))  

Note: *Clicking on the above button kills the kernel.*

The widget sends a custom method on click that is not handled correctly.

In [ ]:
(let [min 0
      max 100
      value 30
      step 1
      w (ipy/play {:value value :min min :max max :step step :interval 500})
      prg (ipy/int-progress {:min min :max max :value value})]
  (.watch w :key0 (fn [_ _ _ new-state] (swap! prg assoc :value (:value new-state))))
  (ipy/h-box {:children [w prg]}))

Note: *As per [example](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Play-(Animation)-widget).
Currently, we are missing the jslink function to link two widgets.*

In [ ]:
(let [w (ipy/date-picker {:description "Pick a date" :disabled false})]
  (alpha/interactive (ipy/label) (mapper identity) {:arg w}))

In [ ]:
(let [w (ipy/color-picker {:description "Pick a color" :value "blue" :concise false})]
  (alpha/interactive (ipy/label) (mapper identity) {:arg w}))

In [ ]:
(def fu (ipy/file-upload))
fu

Looking at the contents of fu atom we don't see any obvious way to access the contents of the uploaded file, only the metadata.

In [ ]:
@fu

In [ ]:
(ipy/controller)

Not tested: dom-widget, directional-link,  grid-box, layout, link, progress-style, slider-style, widget-display-data